## Example of managed attributes via properties

In [1]:
class Person:
    def __init__(self, name) -> None:
        self.name = name

    # Getter function
    @property
    def name(self):
        return self._name

    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected a string, but got type {type(value).__name__.upper()}')
        self._name = value

    @name.deleter
    def name(self):
        raise AttributeError("Cannot delete attribute")

In [2]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name

    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

    @name.deleter
    def name(self):
        print('Deleting a name')
        super(SubPerson, SubPerson).name.__delete__(self)

In [3]:
class SubPerson2(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson2, SubPerson2).name.__set__(self, value)

In [4]:
class SubPerson3(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name

In [5]:
a = Person('Benjamin')
a.name

'Benjamin'

In [6]:
a.name = 'Dave'
a.name

'Dave'

In [7]:
try:
    a.name = 42
except TypeError as e:
    print(e)

Expected a string, but got type INT
